In [ ]:
import numpy as np
import gensim
import nltk
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from queue import PriorityQueue
import math
%load_ext autoreload

In [ ]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)  
stop_words =  set(stopwords.words('english'))

In [ ]:
def get_sentences(path):
    with open(path) as file:
        lines = file.read().replace('\n', '')
    
    return nltk.sent_tokenize(lines)

def filter_sentence(sentence):
    tokens = gensim.utils.simple_preprocess(sentence)#nltk.tokenize.word_tokenize(sentence)
    filtered = [w.lower() for w in tokens if not w in stop_words and w in word_model.vocab]
    return filtered

def get_sentence_embedding(sentence):
    vectors = np.zeros((len(sentence),300))
    for i, word in enumerate(sentence):
        embedding = word_model[word]
        assert np.all(np.isfinite(embedding))
        vectors[i] = embedding
    mean = np.mean(vectors, axis=0)
    return mean


In [ ]:
def embed_text(path):
    sentences = get_sentences(path)
    embeddings = []
    for s in sentences:
        filtered = filter_sentence(s)
        if not len(filtered)==0:
            embedded = get_sentence_embedding(filtered)
            embeddings.append(embedded)
    
    embeddings_array = np.zeros((len(embeddings), 300))
    for i,e in enumerate(embeddings):
        embeddings_array[i] = e
    return embeddings_array

In [ ]:
def reduce_embeddings(embeddings, components=150):
    
    covar =  PCA(n_components=components)
    reduced = covar.fit_transform(embeddings)
    return reduced

In [ ]:
def mean_vector(vectors):
    #vectors are in a list
    assert not len(vectors)==0
    
    first = np.zeros(vectors[0].shape)
    for v in vectors:
        first+=v
    return first/(len(vectors))

def mean_distances(matrix):
    
    total = 0
    num = matrix.shape[0]*(matrix.shape[0] - 1) / 2 
    
    for i in range(matrix.shape[0]):

        for j in range(i, matrix.shape[0]):
            dist = np.sum((matrix[i] - matrix[j])**2)
            total += dist
    
    return total/num

def get_representatives(clusters, cluster_groups):
    
    representatives = []
    
    for i, cluster in enumerate(clusters):
        
        min_dist = np.inf
        index = -1
        for j, v in enumerate(cluster_groups[i]):
            dist = np.sum((v-cluster)**2)
            if dist < min_dist:
                min_dist=dist
                index = j
        representatives.append(cluster_groups[i][index])
    return representatives
    


In [ ]:
def kmeans_clustering(vectors, num_clusters=10, iterations=10):
    
    #initialize clusters to random points
    clusters = vectors[np.random.choice(vectors.shape[0], num_clusters, replace=False)]
    new_clusters = clusters
    for j in range(iterations):
        clusters = new_clusters
        # list of vectors belonging to each cluster
        cluster_groups = [[] for cluster in clusters]
        for i in range(vectors.shape[0]):
            vector = vectors[i]

            min_dist = np.inf
            index = -1
            for i,cluster in enumerate(clusters):
                dist = np.sum( (vector-cluster)**2)
                if dist < min_dist:
                    min_dist = dist 
                    index = i
            
            # add to group of closest cluster
            cluster_groups[index].append(vector)
        print("Iteration:", j, [len(c) for c in cluster_groups])
        new_clusters = [mean_vector(group) for group in cluster_groups]
        
        diff = [new for i,new in enumerate(new_clusters) if not np.array_equal(new, clusters[i])]
        if len(diff)==0:
            break
    return clusters, cluster_groups

In [ ]:
# fantastically slow implmentation, mainly due to
# vectors not being hashable, and looking 
def dbscan_clustering(vectors, min_points=4, radius=0.5):
    # normalize for densities
    mean=np.mean(vectors)
    std=np.std(vectors)
    vectors = (vectors-mean)/std
    
    indices = set(range(vectors.shape[0]))

    pq = PriorityQueue()
    
    # dict of index: [indices] representing cluster relations
    
    # O(n^2)
    cluster_groups = {}
    while not len(indices)==0:
        for e in indices:
            break
        cluster_groups[e] = [e]
        pq.put(e)
        while not pq.empty():
            index = pq.get()
            indices.remove(index)
            vector = vectors[index]
            
            near = []
            for other in indices:
                if other in cluster_groups[index]:
                    continue
                
                other_vec = vectors[other]
                dist = np.sum((vector - other_vec)**2)
                if dist < radius ** 2:
                    # mark visited
                    cluster_groups[index].append(other)
                    cluster_groups[other] = cluster_groups[index] # watch for pointing to the right place
                    near.append(other)
            if len(near) >= min_points - 1: #include the point that put it in the queue
                for n in near:
                    pq.put(n)
    # O(n^2)       
    groups = []
    for index, cluster in cluster_groups.items():
        same = [g for g in groups if set(g)==set(cluster)]
        if len(same)==0:
            groups.append(cluster)
    return [ [vectors[i]*std+mean for i in g] for g in groups  ]

In [ ]:
embeddings = reduce_embeddings(embed_text("bible.txt"), components=150)

In [ ]:
clusters, groups = kmeans_clustering(embeddings, num_clusters=20, iterations=40)
reps = get_representatives(clusters, groups)

In [ ]:
print(len(embeddings))
groups = dbscan_clustering(embeddings,min_points=4, radius=15)
print(mean_distances(embeddings))

In [ ]:
print(len(groups)) # number of clusters from dbscan